# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. 

<b> Цель проекта </b>- построить модель для определения стоимости. 

Нам предоставлены исторические данные: технические характеристики, комплектации и цены автомобилей. 

<b> Описание данных </b>

Признаки:

- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак:

- Price — цена (евро)

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

<b> Задачи проекта: </b>

- Подготовить данные.
- Обучить разные модели, одна из которых — LightGBM, как минимум одна — не бустинг. 
- Проанализировать время обучения, время предсказания и качество моделей.
- Опираясь на критерии заказчика, выбрать лучшую модель, проверить её качество на тестовой выборке.

## Загрузка необходимых библиотек

In [1]:
pip install category_encoders

In [2]:
import time
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Подготовка данных

In [3]:
server_path = '/datasets/autos.csv'   
local_path = 'autos.csv'               
try:
    df = pd.read_csv(server_path)   
except: 
    df = pd.read_csv(local_path)  

In [4]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Заполним пропуски:

Категориальные признаки gearbox и repaired являются бинарными. Заполним пропущенные значения методом np.random.choice, а в колонках vehicle_type, model, fuel_type дополнительной категорией unknown.

In [6]:
df[["Gearbox", "Repaired"]] = df[["Gearbox", "Repaired"]].fillna({
    "Gearbox": np.random.choice(df["Gearbox"].dropna()),
    "Repaired": np.random.choice(df["Repaired"].dropna())
})
df["VehicleType"] = df["VehicleType"].fillna('unknown')
df["Model"] = df["Model"].fillna('unknown')
df["FuelType"] = df["FuelType"].fillna('unknown')


In [7]:
objects_columns = ["VehicleType","Gearbox","Model","FuelType","Brand","Repaired"]
for column in objects_columns:
    print(column, pd.Series(df[column].unique()).str.lower().duplicated().sum())

VehicleType 0
Gearbox 0
Model 0
FuelType 0
Brand 0
Repaired 0


Проверим дубликаты:

In [8]:
df.duplicated().sum()

5

In [9]:
df = df.drop_duplicates()
df.duplicated().sum()

0

In [10]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354364.000000,354364.000000,354364.000000,354364.000000,354364.000000,354364.0,354364.000000
mean,4416.655608,2004.234471,110.093723,128211.373051,5.714641,0.0,50508.461698
std,4514.159690,90.228593,189.851590,37905.136957,3.726433,0.0,25783.124276
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49409.500000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Удалим лишние столбцы:

In [11]:
orig_df = df.copy()
df = df.drop(["DateCrawled","DateCreated","LastSeen","NumberOfPictures","PostalCode", "RegistrationMonth"],axis = 1)

Можно заметить, что в столбце RegistrationYear присутствуют аномальные значения. Установим границы, в которых может находится значение этого столбца: актуальность базы - 2016 год. Объявления "младше" этой даты - ошибки. Сохраним только те машины, чей год регистрации находится в диапазоне с 1960 по 2016 гг.

В Power присутствуют аномальные значения (20 000 лошадиных сил), ограничим их мощностью БелАЗ(а), т.е. 3500.

Столбец Kilometer имеет достаточно реальные значения.

Значения из столбца Price обладают высоким стандартным отклонением, что будет влиять на работу алгоритма.

Такие признаки как NumberOfPictures и PostalCode не влияют на реальную стоимость автомобиля.

Обработаем выбросы и границы значений в упомянутых выше признаках:

In [12]:
df = df.query('RegistrationYear >= 1960 and RegistrationYear <= 2016')
df.loc[df['Power'] > 3500, 'Power'] = 3500

In [13]:
def remove_emissions(df,column):
    q25=np.array(df[column].quantile(0.25))
    q75=np.array(df[column].quantile(0.75))
    first_part=q25-1.5*(q75-q25)
    second_part=q75+1.5*(q75-q25)
    del_index = []
    for index_value, value in zip(df[column].index,df[column]):
        if second_part <= value or value <= first_part:
            del_index.append(index_value)
    
    print('Количество строк, выбранных для удаления ' + str(column)+":",len(del_index))
    return del_index

In [14]:
array_num_col = ["Price","Power"]
count = 0 
for column in array_num_col:
    index_del = remove_emissions(df,column)
    count += len(index_del)
    df = df.drop(index_del,axis = 0)
print("Было удалено:", count)

Количество строк, выбранных для удаления Price: 17405
Количество строк, выбранных для удаления Power: 5764
Было удалено: 23169


In [15]:
df.duplicated().sum()

49866

In [16]:
df = df.drop_duplicates()
df.duplicated().sum()

0

In [17]:
df.corr()

,Price,RegistrationYear,Power,Kilometer
Price,1.000000,0.438512,0.420075,-0.278994
RegistrationYear,0.438512,1.000000,0.131584,-0.175357
Power,0.420075,0.131584,1.000000,0.149844
Kilometer,-0.278994,-0.175357,0.149844,1.000000


Наиболее коррелирующим с целевым признаком является признак отвечающий за мощность.

## Обучение моделей

Разделим и закодируем выборки:

In [18]:
var_numeric = ['RegistrationYear', 'Power', 'Kilometer']
var_categorical = ['VehicleType', 'FuelType', 'Gearbox', 'Repaired']
var_me = ['Model', 'Brand']

In [19]:
df_light = df.copy()
df_light = df_light.astype({"VehicleType":'category',
                                  "Gearbox":'category',
                                  "Model":'category',
                                  "FuelType":'category',
                                  "Brand":'category',
                                  "Repaired":'category'})
target_light = df_light['Price']
features_light = df_light.drop('Price', axis=1)

features_train_light, features_val, target_train_light, target_val = train_test_split(features_light, target_light, test_size=0.4, 
                                                                              random_state=42)
features_valid_light, features_test_light, target_valid_light, target_test_light = train_test_split(features_val, target_val, test_size=0.5, 
                                                                              random_state=42)

print(features_train_light.shape, features_valid_light.shape, features_test_light.shape)
print(target_train_light.shape, target_valid_light.shape, target_test_light.shape)

(159826, 9) (53275, 9) (53276, 9)
(159826,) (53275,) (53276,)


In [20]:
target = df['Price']
features = df.drop('Price', axis=1)


features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, 
                                                                              random_state=42)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, 
                                                                              random_state=42)


scaler = StandardScaler(with_mean=False)
features_train[var_numeric] = scaler.fit_transform(features_train[var_numeric])
features_valid[var_numeric] = scaler.transform(features_valid[var_numeric])
features_test[var_numeric] = scaler.transform(features_test[var_numeric])

enc = ce.MEstimateEncoder()
features_train[var_me] = enc.fit_transform(features_train[var_me], target_train)
features_valid[var_me] = enc.transform(features_valid[var_me])
features_test[var_me] = enc.transform(features_test[var_me])

encoder = OneHotEncoder(handle_unknown="ignore")
features_train = encoder.fit_transform(features_train, target)
features_valid = encoder.transform(features_valid)
features_test = encoder.transform(features_test)



print(features_train.shape, features_valid.shape, features_test.shape)
print(target_train.shape, target_valid.shape, target_test.shape)

<ipython-input-20-3fa5ccec14fa>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[var_numeric] = scaler.fit_transform(features_train[var_numeric])
C:\Users\js\anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\js\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

(159826, 627) (53275, 627) (53276, 627)
(159826,) (53275,) (53276,)


Обучим модели:

In [21]:
%%time
# LightGBM
lgb_train = lgb.Dataset(features_train_light, target_train_light)
lgb_valid = lgb.Dataset(features_valid_light, target_valid_light, reference=lgb_train)
parameters_light = {'metric': 'l2', 'max_depth':10,"random_state": 42,"learning_rate":0.1}
light = lgb.train(parameters_light,
                lgb_train,
                num_boost_round=1000,
                valid_sets=[lgb_train, lgb_valid],
                verbose_eval=100)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005748 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 621
[LightGBM] [Info] Number of data points in the train set: 159826, number of used features: 9
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 3800.007915


C:\Users\js\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\js\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[100]	training's l2: 2.03331e+06	valid_1's l2: 2.21252e+06
[200]	training's l2: 1.90046e+06	valid_1's l2: 2.14991e+06
[300]	training's l2: 1.82295e+06	valid_1's l2: 2.12989e+06
[400]	training's l2: 1.76485e+06	valid_1's l2: 2.11546e+06
[500]	training's l2: 1.7166e+06	valid_1's l2: 2.10701e+06
[600]	training's l2: 1.67359e+06	valid_1's l2: 2.09996e+06
[700]	training's l2: 1.64075e+06	valid_1's l2: 2.09629e+06
[800]	training's l2: 1.61082e+06	valid_1's l2: 2.09257e+06
[900]	training's l2: 1.58649e+06	valid_1's l2: 2.09057e+06
[1000]	training's l2: 1.56201e+06	valid_1's l2: 2.08722e+06
Wall time: 3.34 s


In [22]:
rmse_lgb = mean_squared_error(target_valid_light, light.predict(features_valid_light), squared=False)
print("RMSE LightGBM на валидационной выборке: %.2f" % rmse_lgb)

RMSE LightGBM на валидационной выборке: 1444.72


In [23]:
%%time
#CatBoost
catboost = CatBoostRegressor(learning_rate=0.1, depth=10,
                                   loss_function='RMSE',
                                   random_state=42)

catboost.fit(features_train_light, target_train_light, eval_set=(features_valid_light, target_valid_light), cat_features=["VehicleType","Gearbox",
                                                             "Model","FuelType",
                                                             "Brand","Repaired"], verbose=100, early_stopping_rounds=50)


0:	learn: 3304.2287771	test: 3279.3465755	best: 3279.3465755 (0)	total: 415ms	remaining: 6m 54s
100:	learn: 1444.3063374	test: 1509.4948993	best: 1509.4948993 (100)	total: 20.4s	remaining: 3m 1s
200:	learn: 1378.0276733	test: 1480.6692376	best: 1480.6599557 (199)	total: 40.8s	remaining: 2m 42s
300:	learn: 1338.1391028	test: 1470.8549227	best: 1470.8549227 (300)	total: 1m 3s	remaining: 2m 26s
400:	learn: 1311.1080941	test: 1466.0434978	best: 1466.0434978 (400)	total: 1m 25s	remaining: 2m 8s
500:	learn: 1288.4823117	test: 1462.6472549	best: 1462.6472549 (500)	total: 1m 48s	remaining: 1m 48s
600:	learn: 1268.3477187	test: 1460.2414305	best: 1460.1852637 (599)	total: 2m 14s	remaining: 1m 29s
700:	learn: 1252.1492478	test: 1458.9775287	best: 1458.9716831 (699)	total: 2m 40s	remaining: 1m 8s
800:	learn: 1237.5464308	test: 1457.5272681	best: 1457.5230506 (798)	total: 3m 7s	remaining: 46.5s
900:	learn: 1222.0439891	test: 1456.0677912	best: 1456.0677912 (900)	total: 3m 34s	remaining: 23.6s
999:

In [24]:
rmse_catboost = mean_squared_error(target_valid_light, catboost.predict(features_valid_light), squared=False)
print("RMSE CatBoostRegressor на валидационной выборке: %.2f" % rmse_catboost)

RMSE CatBoostRegressor на валидационной выборке: 1455.67


In [25]:
%%time
# RandomForest
RF = RandomForestRegressor(criterion = "mse",random_state=42,n_estimators = 100,n_jobs=-1)
RF.fit(features_train,target_train)

Wall time: 17min 42s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [26]:
rmse_forest = mean_squared_error(target_valid, RF.predict(features_valid), squared=False)
print("RMSE RandomForestRegressor на валидационной выборке: %.2f" % rmse_forest)


RMSE RandomForestRegressor на валидационной выборке: 1600.90


Посчитаем ошибку для каждой модели на тестовой выборке и определим время предсказаний:

In [27]:
%%time
print("LightGBM:",(mean_squared_error(target_test_light,light.predict(features_test_light),squared=False)))

LightGBM: 1445.9829057027985
Wall time: 1.77 s


In [28]:
%%time
print("CatBoost:",mean_squared_error(target_test_light,catboost.predict(features_test_light),squared=False))

CatBoost: 1448.8608042202675
Wall time: 1.07 s


In [29]:
%%time
print("RandomForest:",mean_squared_error(target_test,RF.predict(features_test),squared=False))

RandomForest: 1600.900704270407
Wall time: 1.7 s


## Анализ моделей

В условии указано, что решающим факторами при выборе модели являются следующие показатели:

Время обучения
Время предсказания
Качество предсказаний

В ходе работы получили следующие результаты:

- быстрее всего обучилась модель LightGBM с показателем 3.34 s(на обучение рандомного леса ушло 17min 42s, CatBoost - 4min 2s);
- время предсказания меньше всего у модели CatBoost (1.07), модель RandomForest по этому показателю LightGBM и RandomForest сработали почти одинаково (1.77 и 1.7 s);
- наименьшую ошибку выдает LightGBM (1445.983), разрыв с CatBoost небольшой (1448.86), RandomForest справилась хуже всего.

Основываясь на данных признаках мы можем убрать из сравнения RandomForest, так как и время ее обучения, и качество уступает двум другим моделям. 

Разница предсказания в 70 ms не критична, а в работе модели всегда важна точность предсказания. 
Основываясь на этом, можно сделать вывод, что заказчику лучше всего подойдет модель построенная на LightGBM.